### TRABAJO PRÁCTICO N° 4 MÉTODOS SUPERVISADOS: REGRESIÓN & CLASIFICACIÓN USANDO LA EHP

#### A. Enfoque de validación

###

In [4]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

class DivisionDatos:
    def __init__(self):
        self.variables_independientes = ['CH04', 'CH06', 'CAT_OCUP', 'PP02H', 'CH08']
        self.datos = None
        
    def cargar_datos(self, archivo="Base_Respondieron.xlsx"):
        self.datos = pd.read_excel(archivo)
        columnas_necesarias = ['ESTADO', 'ANO4'] + self.variables_independientes
        self.datos = self.datos.dropna(subset=columnas_necesarias)
        return self
    
    def preparar_variables(self, df):
        y = (df['ESTADO'] == 'Desocupado').astype(int)
        X = pd.get_dummies(df[self.variables_independientes], drop_first=True)
        X.insert(0, 'intercepto', 1)
        return X, y
    
    def dividir_por_año(self, año):
        df_año = self.datos[self.datos['ANO4'] == año].copy()
        if len(df_año) == 0:
            return None
        
        X, y = self.preparar_variables(df_año)
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.7, random_state=444, stratify=y)
        
        return {
            'X_train': X_train, 'X_test': X_test, 'y_train': y_train, 'y_test': y_test, 'df_año': df_año, 'año': año
        }
    
    def guardar_datasets(self, resultado):
        año = resultado['año']
        
        train_df = resultado['X_train'].copy()
        train_df['y_desocupado'] = resultado['y_train'].values
        test_df = resultado['X_test'].copy()
        test_df['y_desocupado'] = resultado['y_test'].values
        
        train_df.astype(int).to_excel(f'train_data_{año}.xlsx', index=False)
        test_df.astype(int).to_excel(f'test_data_{año}.xlsx', index=False)
        
        df_año = resultado['df_año']
        X_original, y_original = self.preparar_variables(df_año)
        X_train_temp, X_test_temp, _, _ = train_test_split(X_original, y_original, test_size=0.7, random_state=444, stratify=y_original)
        
        df_año.loc[X_train_temp.index].to_excel(f'train_original_{año}.xlsx', index=False)
        df_año.loc[X_test_temp.index].to_excel(f'test_original_{año}.xlsx', index=False)
    
    def procesar_todos_los_años(self):
        for año in sorted(self.datos['ANO4'].unique()):
            resultado = self.dividir_por_año(año)
            if resultado:
                self.guardar_datasets(resultado)

def main():
    DivisionDatos().cargar_datos().procesar_todos_los_años()

if __name__ == "__main__":
    main() 

In [5]:
import pandas as pd
import numpy as np

class AnalisisDiferenciasMedias:
    def __init__(self):
        self.años = [2004, 2024]
        
    def cargar_bases(self, año):
        train_df = pd.read_excel(f'train_data_{año}.xlsx')
        test_df = pd.read_excel(f'test_data_{año}.xlsx')
        return train_df, test_df
    
    def calcular_diferencias_medias(self, train_df, test_df, año):
        variables_x = [col for col in train_df.columns if col != 'y_desocupado']
        
        train_x = train_df[variables_x]
        test_x = test_df[variables_x]
        
        medias_train = train_x.mean()
        medias_test = test_x.mean()
        
        tabla_diferencias = pd.DataFrame({
            'Variable': variables_x,
            'Media_Train': medias_train.values,
            'Media_Test': medias_test.values,
            'Diferencia': (medias_train - medias_test).values,
            'Diferencia_Abs': abs(medias_train - medias_test).values,
            'Año': año
        })
        
        tabla_diferencias = tabla_diferencias.sort_values('Diferencia_Abs', ascending=False)
        return tabla_diferencias
    
    def analizar_todos_los_años(self):
        for año in self.años:
            train_df, test_df = self.cargar_bases(año)
            tabla = self.calcular_diferencias_medias(train_df, test_df, año)
            
            print(f"Año {año}:")
            print(f"  Variables: {len(tabla)}")
            print(f"  Mayor diferencia: {tabla['Diferencia_Abs'].max():.4f}")
            print(f"  Promedio diferencias: {tabla['Diferencia_Abs'].mean():.4f}")
            
            print("  Top 3 mayores diferencias:")
            for i, (_, row) in enumerate(tabla.head(3).iterrows()):
                print(f"    {i+1}. {row['Variable']}: {row['Diferencia']:.4f}")
            print()

analizador = AnalisisDiferenciasMedias()
analizador.analizar_todos_los_años()

Año 2004:
  Variables: 102
  Mayor diferencia: 0.0138
  Promedio diferencias: 0.0019
  Top 3 mayores diferencias:
    1. CH04_Varón: 0.0138
    2. CH08_No paga ni le descuentan: 0.0115
    3. CH08_Obra social (incluye PAMI): -0.0094

Año 2024:
  Variables: 106
  Mayor diferencia: 0.0078
  Promedio diferencias: 0.0018
  Top 3 mayores diferencias:
    1. CH04_Varón: 0.0078
    2. CH08_Obra social (incluye PAMI): 0.0063
    3. CH06_21: -0.0061

